# Vsodeのインストール
お使いのインストーラーをダウンロードしてインストールしてください  
https://code.visualstudio.com/download

# Python Extentionの設定

1. cmd shift p ー＞　「python」をインストール

## ノートブックを開くためには
cmd shift p -> 「cretate ju」をタイプし「create jupyter notebook」を選択します

# Javaのインストール
もしお使いのPCにJavaがインストールされていない場合はJava8か11をインストールお願いします。

# 【Macの場合】
brew update
brew install java11
## ZSHの場合(Bashの場合は~/.bash_profile)
echo 'export PATH="/opt/homebrew/opt/openjdk@11/bin:$PATH"' >> ~/.zshrc  
export CPPFLAGS="-I/opt/homebrew/opt/openjdk@11/include" >> ~/.zshrc  
source ~/.zshrc  

※brewのインストールはこちら
https://brew.sh/  

# 【Windowsの場合】
https://www.oracle.com/java/technologies/downloads/#java11  
でJDKをダウンロードしてください

## setting.jsonの変更
cmd shift pを押して「settings.json」と打ち込みます「Open Setitngs Jsonを開きます」
java.homeを追加します（以下は私の例です）

```
{
    "workbench.colorTheme": "Default Dark+",
    "window.zoomLevel": 3,
    "editor.suggestSelection": "first",
    "vsintellicode.modify.editor.suggestSelection": "automaticallyOverrodeDefaultValue",
    "java.home": "/opt/homebrew/opt/openjdk@11/bin",
}
```

# mysqlのインストール

バージョンは5.7系をインストールします

MacBookの場合
1. brew install mysql  
※brewのインストールは「https://brew.sh/」よりお願いします

2. brew install mysql@5.7
3. brew services start mysql@5.7
4. brew services restart mysql@5.7
5. echo 'export PATH="/opt/homebrew/opt/mysql@5.7/bin:$PATH"' >> ~/.zshrc
5. mysql -uroot
6. ALTER USER 'root'@'localhost' IDENTIFIED BY 'root';   パスワードを設定します

Windowsの場合  
以下のインストーラーをご利用ください
https://dev.mysql.com/downloads/installer/

## インストールがうまく行かない場合(Mac)
brew services stop mysql@5.7

```
sudo rm -rf /opt/homebrew/Cellar/mysql*
sudo rm -rf /opt/homebrew/bin/mysql*
sudo rm -rf /opt/homebrew/var/mysql*
sudo rm -rf /opt/homebrew/etc/my.cnf
sudo rm -rf /opt/homebrew/share/mysql*
sudo rm -rf /opt/homebrew/opt/mysql*
sudo rm -rf /etc/my.cnf

sudo rm -rf /usr/local/Cellar/mysql*
sudo rm -rf /usr/local/bin/mysql*
sudo rm -rf /usr/local/var/mysql*
sudo rm -rf /usr/local/etc/my.cnf
sudo rm -rf /usr/local/share/mysql*
sudo rm -rf /usr/local/opt/mysql*
sudo rm -rf /etc/my.cnf
```

してから再度インストールしてみてください


# Sparkのインストール

MacBookの場合

```
brew install apache-spark  
echo "export SPARK_HOME=/opt/homebrew/opt/apache-spark/libexec/" >> ~/.zshrc  
echo "export "PATH=${PATH}:${SPARK_HOME}/" >> ~/.zshrc  
```

Windowsの場合  

```
以下よりtar.gzをダウンロードし解凍する
https://spark.apache.org/downloads.html
システムの環境変数に
SPARK_HOMEとPATHの設定を行う(SPARK_HOMEは解凍したディレクトリになります)
```

※パスは環境ごとに違う可能性があります。

# もし既に簡易版pysparkが入っている方は削除しておきましょう
pip3 uninstall pyspark --y 

## findsparkのインストール
pip install findspark --y  

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#問題なく実行できればOKです


# Hive MetaStoreの設定
同梱しているhive-site.xmlを設定してMysqlをメタストアにします。  
SPARK_HOMEの位置によって場所は異なりますので適宜切り替えをお願いします。  


## mysqlドライバをインストールします
私の環境では$SPARK_HOME/jars配下への格納です。  
/opt/homebrew/opt/apache-spark/libexec/jars  

URL:  
https://dev.mysql.com/downloads/  
「JDBC Driver for MySQL (Connector/J)」をダウンロードします  

以下のコマンドで解凍します。  
tar -xzvf mysql-connector-java-8.0.27.tar.gz  

以下のファイルがドライバーなので「/opt/homebrew/opt/apache-spark/libexec/jars」直下にファイルを移動します  
「mysql-connector-java-8.0.27.jar」  

## hive-site.xmlの設定
同梱しているhive-site.xmlを「/opt/homebrew/opt/apache-spark/libexec/conf」へ配置します。


In [ ]:
#pysparkに必要なライブラリを読み込む

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("show tables").show() 

# 空の結果が表示されればOKです

# エラーになってしまう場合

## hiveをクローン
https://github.com/apache/hive

## メタストアをオリジナルから作成する
1. cd /Users/saitouyuuki/Desktop/src/hive/metastore/scripts/upgrade/mysql
2. mysql -uroot -proot
3. use metastore
4. source hive-schema-2.3.0.mysql.sql
5. source hive-txn-schema-2.3.0.mysql.sql
場合によってはdrop database metastoreを実行したのちcreate database metastoreを実行してみてください。